# Working with In-Memory OLTP
In this demo we'll do following steps
1. Database configuration for In-Memory OLTP
2. Table and indexes creation
3. Creation of natively compiled stored procedure
4. Performance test
5. Memory consumption diagnostics

## Database configuration for In-Memory OLTP
Database which contains memory optimized tables needs to have memory optimized filegroup. The filegroup is used by SQL Server for memory dumps during checkpoint. It ensures full durability of memory optimized tables.

In [ ]:
use master
GO

alter database Demo add filegroup INMEM contains memory_optimized_data
go

In [ ]:
use master
go

alter database Demo add file
(
    name = 'InMemData'
    , filename = 'F:\InMemFs\DemoInMem'
) to filegroup INMEM
go

Now, the database is prepared for memory optimized workload.

## Creating memory optimized table and indexes

In [ ]:
use Demo
go

create table dbo.SensorData
(
    Id int not null identity 
        constraint pk_SensorData primary key nonclustered hash with (bucket_count = 2097152) -- hash index
    , CurrentTime datetime2(7) not null constraint df_CurrentTime default (sysdatetime())
    , TheValue dec(10, 3) not null constraint df_TheValue default (rand() * 1000)
    , index ix_SensorData_TheValue (TheValue) -- range index
)
with
(
    memory_optimized = on       -- filegroup is not explicitly set
    , durability = schema_and_data
)
go

## Natively compiled stored procedure

In [ ]:
use Demo
go

create or alter proc dbo.procGenerateSensorData
with NATIVE_COMPILATION, SCHEMABINDING, execute as caller
AS
begin atomic with (transaction isolation level = snapshot, language = 'english')
    declare @i int = 1
    while @i <= 1000000
     BEGIN
        insert dbo.SensorData default values
        set @i+=1
     end
end
go

In [ ]:
use Demo
GO

set statistics io, time ON
exec dbo.procGenerateSensorData
set statistics io, time OFF
go

## Memory consumption by memory optimized objects

In [ ]:
use Demo
GO
select count(*) from dbo.SensorData

select
    memory_consumer_desc
    , allocated_bytes / 1024./1024. as allocated_mb
    , used_bytes / 1024. / 1024. as used_mb
from sys.dm_db_xtp_memory_consumers where object_id = OBJECT_ID('dbo.SensorData')
go